In [34]:
import wandb
import matplotlib.pyplot as plt
import scipy
import numpy as np
import json
import os
from collections import defaultdict
from copy import deepcopy

plt.rcParams["axes.grid"] = False

import sys
sys.path.append('..')

In [35]:
language = 'german'

sys.argv = [
"--device cuda",
"--data-folder", "..\\data",
"--seed", "13",
"--configuration", "char-to-char",
"--language", language,
"--challenge", "named-entity-recognition"]

In [36]:
# Configure container:
from dependency_injection.ioc_container import IocContainer

container = IocContainer()

In [37]:
plot_service = container.plot_service()
file_service = container.file_service()

In [38]:
entity = 'eval-historical-texts'
project = 'named-entity-recognition'

In [39]:
def get_wandb_runs(run_filters: dict):
    api = wandb.Api()
    runs = api.runs(path=f'{entity}/{project}', filters=run_filters)
    return runs

In [40]:
main_config = {
    'createdAt': {
        '$gt': '20200710000000'
    },
    'state': 'finished',
}

available_filters = {
    'multi-task-document': {
        'config.split_type': 'document',
        'config.entity_tag_types': ["literal-fine","literal-coarse","metonymic-fine","metonymic-coarse","component","nested"]
    },
    'multi-task-multi-segment': {
        'config.split_type': 'multi-segment',
        'config.entity_tag_types': ["literal-fine","literal-coarse","metonymic-fine","metonymic-coarse","component","nested"]
    },
    'single-task-document': {
        'config.split_type': 'document',
        'config.entity_tag_types': {
            '$ne': ["literal-fine","literal-coarse","metonymic-fine","metonymic-coarse","component","nested"]
        }
    },
    'single-task-multi-segment': {
        'config.split_type': 'multi-segment',
        'config.entity_tag_types': {
            '$ne': ["literal-fine","literal-coarse","metonymic-fine","metonymic-coarse","component","nested"]
        }
    },
}

In [43]:
runtime_per_conf = {
    unique_str: 0.0 for unique_str in available_filters.keys()
}

conf_runs = {
    unique_str: 0 for unique_str in available_filters.keys()
}

for unique_str, available_filter in available_filters.items():
    run_filter = deepcopy(main_config)
    run_filter.update(available_filter)
    runs = get_wandb_runs(run_filter)
    
    for run in runs:
        runtime = float(run.summary['_runtime'])
        runtime_per_conf[unique_str] += runtime
        conf_runs[unique_str] += 1
    # break

In [50]:
print(conf_runs)
print(runtime_per_conf)

for unique_str in available_filters.keys():
    print(unique_str)
    result = runtime_per_conf[unique_str] / conf_runs[unique_str]
    print(f' - {round(result / 60, 2)} minutes')
    print(f' - {round(result / 3600, 2)} hours')

{'multi-task-document': 23, 'multi-task-multi-segment': 124, 'single-task-document': 12, 'single-task-multi-segment': 13}
{'multi-task-document': 478994.86743712425, 'multi-task-multi-segment': 1212422.9030425549, 'single-task-document': 103785.37310910225, 'single-task-multi-segment': 20951.599126577377}
multi-task-document
 - 347.1 minutes
 - 5.78 hours
multi-task-multi-segment
 - 162.96 minutes
 - 2.72 hours
single-task-document
 - 144.15 minutes
 - 2.4 hours
single-task-multi-segment
 - 26.86 minutes
 - 0.45 hours
